In [1]:
import duckdb
from pathlib import Path
import datetime

In [2]:
ts = datetime.datetime.now()

In [3]:
base_path = Path().cwd().parent
source_path = base_path / Path('raw/entity/anime-character')
target_path = base_path / Path(f'silver/anilist/voice-actor/voice-actor-{ts.year}-{ts.month}-{ts.day}.parquet')

In [6]:
tb_voice_actor = duckdb.sql(f"""
    WITH source AS(
        SELECT * FROM read_json_auto('{str(source_path)}/*.json')
    )
    , tb_edge_list AS (
        SELECT 
        json_extract(source, 
        [
            '$.data.Media.id' 
            , '$.data.Media.characters.edges'
        ]) edges_list
        FROM source
    )
    , tb_unnest_edge_list AS (
        SELECT 
            edges_list[1] AS anime_id
        ,   unnest(edges_list[2]::JSON[]) as edge 
        FROM tb_edge_list 
    )
    , tb_extract_edge AS (
        SELECT
        anime_id
        , json_extract(edge, [
            '$.role'
            , '$.voiceActors'
        ]) edges
        FROM tb_unnest_edge_list
    ), unnest_edge_voiceactor AS(
        SELECT
        anime_id as anime_id
        , edges[1] AS role
        , json_extract(unnest(edges[2]::JSON[]),[
            'id'
            , '$.name.first'
            , '$.name.last'
        ]) AS voiceActors
        FROM tb_extract_edge 
    ), rename AS(
        SELECT 
        anime_id
        , role
        , voiceActors[1] id
        , voiceActors[2] name_first
        , voiceActors[3] name_last
        FROM unnest_edge_voiceactor
    ), cast_variable AS(
    SELECT  
        id::INT AS id
        ,name_first::VARCHAR AS name_first
        ,name_last::VARCHAR AS name_last
        ,anime_id::INT AS anime_id
        ,role::VARCHAR AS role
    FROM rename
    )
    SELECT *
    FROM cast_variable
""")

In [7]:
tb_voice_actor.to_parquet(str(target_path))